<p><font size="5"><b>PROJET AIRBNB - SIMPLON DEV. DATA : PYTHON - WORDCLOUD </b></font></p>


<p><font size="4"><b> *Analyse données Airbnb Paris - Mai 2020*</b></font></p>

<p><font size="3"> *Source données : http://insideairbnb.com/get-the-data.html* </font></p>

*© 2020, Gérald Bouget  (<mailto:geraldbouget@gmail.com>)

___

In [13]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_items', None)

import matplotlib.pyplot as plt
%matplotlib inline

import re

from stop_words import get_stop_words
stop_words = get_stop_words('english')
stop_words_fr = get_stop_words('french')
from stop_words import safe_get_stop_words
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
import itertools
import collections
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual,FloatSlider, Layout, Style


In [3]:
# df reviews
df_reviews = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2020-05-10/data/reviews.csv.gz', 
               compression='gzip')

In [16]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2577,366217274,2019-01-02,28047930,Kate,Beautiful apartment in a really handy location...
1,3109,123127969,2016-12-27,12389804,Sophie,The host canceled this reservation the day bef...
2,3109,123274144,2016-12-28,67553494,Tom'S,The host canceled this reservation 2 days befo...
3,3109,207127433,2017-10-28,51636494,Patricia,Tout s'est bien déroulé. Merci bien. PG
4,3109,208779822,2017-11-03,4142888,Patricia,Un petit nid fouiller douillet situé dans app...


In [5]:
# df listing
df_listing_full = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2020-05-10/data/listings.csv.gz',
               compression='gzip', low_memory=False)

In [20]:
df_listing_full.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2577,https://www.airbnb.com/rooms/2577,20200510041557,2020-05-12,Loft for 4 by Canal Saint Martin,"100 m2 loft (1100 sq feet) with high ceiling, ...",The district has any service or shop you may d...,"100 m2 loft (1100 sq feet) with high ceiling, ...",none,NaN,NaN,"Metro lines : 4, 8, 3, 11, 9.",NaN,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/09da057c-0...,NaN,2827,https://www.airbnb.com/users/show/2827,Karine,2008-09-09,"Casablanca, Grand Casablanca, Morocco","Arrivée à Casablanca il y a 4 ans, je suis amo...",within a day,100%,100%,f,https://a0.muscache.com/im/pictures/user/17fe6...,https://a0.muscache.com/im/pictures/user/17fe6...,République,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Paris, Île-de-France, France",République,Entrepôt,NaN,Paris,Île-de-France,75010,Paris,"Paris, France",FR,France,48.86993,2.36251,t,Loft,Entire home/apt,4,2.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,Heating,Washer,Dryer...",NaN,$125.00,NaN,NaN,$0.00,$40.00,1,$0.00,3,1125,3,3,1125,1125,3.0,1125.0,18 months ago,t,30,60,90,365,2020-05-12,1,0,2019-01-02,2019-01-02,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,NaN,"{""translation missing: en.occupancy.taxes.juri...",t,f,strict_14_with_grace_period,f,f,1,1,0,0,0.06
1,3109,https://www.airbnb.com/rooms/3109,20200510041557,2020-05-13,zen and calm,Appartement très calme de 50M2 Utilisation de ...,I bedroom appartment in Paris 14,I bedroom appartment in Paris 14 Good restaura...,none,Good restaurants very close the Montparnasse S...,NaN,RER B Metro Ligne 13 Pernety Metro Ligne 13 Pl...,"A la demande, vous pouvez avoir accès à la cha...",yes I can help you out,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/baeae9e2-c...,NaN,3631,https://www.airbnb.com/users/show/3631,Anne,2008-10-14,"Paris, Île-de-France, France",NaN,within a day,100%,50%,f,https://a0.muscache.com/im/users/3631/profile_...,https://a0.muscache.com/im/users/3631/profile_...,Alésia,1.0,1.0,"['email', 'phone', 'facebook', 'reviews']",t,f,"Paris, Île-de-France, France",Alésia,Observatoire,NaN,Paris,Île-de-France,75014,Paris,"Paris, France",FR,France,48.83349,2.31852,f,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{Internet,Wifi,Kitchen,""Paid parking off premi...",NaN,$60.00,$490.00,NaN,$150.00,$60.00,1,$0.00,

In [6]:
df_listing_utile = df_listing_full[['id', 'neighbourhood_cleansed']]

In [7]:
# merge entre les 2 df sur id
df = pd.merge(df_reviews, df_listing_utile.rename(columns={'id':'listing_id'}))

In [18]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,neighbourhood_cleansed
0,2577,366217274,2019-01-02,28047930,Kate,Beautiful apartment in a really handy location...,Entrepôt
1,3109,123127969,2016-12-27,12389804,Sophie,The host canceled this reservation the day bef...,Observatoire
2,3109,123274144,2016-12-28,67553494,Tom'S,The host canceled this reservation 2 days befo...,Observatoire
3,3109,207127433,2017-10-28,51636494,Patricia,Tout s'est bien déroulé. Merci bien. PG,Observatoire
4,3109,208779822,2017-11-03,4142888,Patricia,Un petit nid fouiller douillet situé dans app...,Observatoire


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1363744 entries, 0 to 1363743
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   listing_id              1363744 non-null  int64 
 1   id                      1363744 non-null  int64 
 2   date                    1363744 non-null  object
 3   reviewer_id             1363744 non-null  int64 
 4   reviewer_name           1363744 non-null  object
 5   comments                1363237 non-null  object
 6   neighbourhood_cleansed  1363744 non-null  object
dtypes: int64(3), object(4)
memory usage: 83.2+ MB


In [8]:
# liste quartiers
df.neighbourhood_cleansed.unique()

array(['Entrepôt', 'Observatoire', 'Hôtel-de-Ville', 'Opéra',
       'Popincourt', 'Buttes-Montmartre', 'Élysée', 'Gobelins',
       'Luxembourg', 'Buttes-Chaumont', 'Louvre', 'Palais-Bourbon',
       'Reuilly', 'Bourse', 'Ménilmontant', 'Vaugirard',
       'Batignolles-Monceau', 'Panthéon', 'Passy', 'Temple'], dtype=object)

In [15]:
# AVERISSEMENT : PROCESS POUVANT PRENDRE UN CERTAIN TEMPS...

# pour avoir toute la longueur du texte avant la box
style = {'description_width': 'initial'}

@interact(quartier=widgets.Dropdown(options=df.neighbourhood_cleansed.unique(), description='Choisir un quartier',
                                style=style))
def review_wordcloud(quartier):
    # selection quartier Hotel de Ville
    df_q = df[df.neighbourhood_cleansed==quartier]
    
    # selection comments only
    df_q_comments=df_q[['comments']]
    
    # conversion colonne comments en str
    df_q_comments=df_q_comments.astype(str)
    
    # nettoyage texte
    df_q_comments_clean=df_q_comments.iloc[:,0].apply(lambda x: re.sub('[\r\n\,\:\.\;\!\d\(\)]', "", x)).apply(
            lambda x: x.lower().split())
    
    # conversion de tous les comments en liste de mots
    #entre crochets= list comprehension pour creer liste de tuple de mots
    #itertools pour iterer dans tuple et creer liste de mots seuls
    liste_mots_q = list(itertools.chain(*([values for key, values in df_q_comments_clean.items()])))
    
    # contenation des listes stopword francais et anglais
    liste_stop_word_module_en = stop_words
    liste_stop_word_module_fr = stop_words_fr
    liste_stop_word_def = liste_stop_word_module_en + liste_stop_word_module_fr
    
    # liste stopword perso (à mettre à jour soi-même)
    liste_perso_stop_words = ['The', 'This', 'paris', 'Paris', 'place', 'host', 'metro', 'appartement', 'apartment', 
    'location''stay', 'us', 'time', 'minutes', 'back', 'also', 'will', 'séjour', 'area', 'location', 'stay', 'one', 'two',
    'walk', 'situé', 'can', 'home', 'flat', 'stayed', 'get', 'just', 'muy', 'l\'appartement', 'night', 'day',
    'days', 'located', 'el', 'bed', 'check', 'airbnb', 'quartier', 'next', 'e', 'bathroom', 'needed', 'thank', 'you',
    'around', 'neighbourhood', 'studio', 'like', 'can', 'need', 'walk', 'station', 'neighborhood', 'right', 'lots', 
    'definitly','room', 'restaurnats', 'people', 'many', 'room', 'within', 'everything',
    'staying', 'walking', 'plus', 'made', 'kitchen', 'plus', 'logement', 'quite', 'definitly', 'métro','much',
    'even', 'arrival', 'accueil', 'feel', 'building', 'family', 'nearby', 'distance', 'para', 'quick', '-', 'door',
    'trip', 'go', 'lot', 'first', 'felt', 'bit', '&', 'make', 'short', 'near', 'questions', 'provided', 'extremely',
    'met', 'kind', 'access', 'gave', 'etc', 'con', 'arrived', 'it\'s', 'plenty', 'exactly', 'things', 'always',
    'dame', 'little', 'away', 'shower', 'stairs', 'windows', 'felt', '&', 'really']
    
    # ajout list perso à list stop word
    for stopword in liste_perso_stop_words:
        liste_stop_word_def.append(stopword)
        
    # nouvelle liste de mots sans les stop words
    #set_liste_stop_word = set(liste_stop_word_def)
    #df_q_comments_def=list(set(liste_mots_q) - set(liste_stop_word_def))
    df_q_comments_def=[x for x in liste_mots_q if x not in liste_stop_word_def]
    
    # liste des mots les plus utlisés (les 100 premiers)
    wordlist_clean_distr_count = collections.Counter(df_q_comments_def)
    cent_premiers_mots=wordlist_clean_distr_count.most_common(200)
    
    # wordcloud
    # CONVERSION DES DONNEES EN DICTIONNAIRE UTILISABLE PAR WORDCLOUD
    dict_wordcloud = dict(cent_premiers_mots)

    # Create and generate a word cloud image:
    wordcloud = WordCloud(max_font_size=100, max_words=150, background_color="white", width=800, height=400).fit_words(
    dict_wordcloud).to_file(f'{quartier}.png')  # utilisation methode fitword pour prendre en compte le nombre de mots du dic

    # Display the generated image:
    plt.figure(figsize=(15, 15))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    return plt.show()

interactive(children=(Dropdown(description='Choisir un quartier', options=('Entrepôt', 'Observatoire', 'Hôtel-…